In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
#Loading data as df (dataframe)
df = pd.read_excel('Investor Report-06_21_2025.xlsx', sheet_name=0)
df.head(2)

,Case Name,Account Name,State,Rate,Provider Client,DOL,Subject,Name of Procedure,Case Stage,Stage Chg Dt,...,Inv 1 Pd Dt,Inv 1 Pd $,Inv 2 Pd Dt,Inv 2 Pd $,Inv 3 Pd Dt,Inv 3 Pd $,Funding Status,Case Notes,Lien Notes,Case Status
0,Kami Orton,ALLRED VOGT & STUART,NaN,NaN,Durango Surgery Center,NaN,Surgery 1,NaN,Omni Purchase,09/21/2023,...,NaN,0.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,Case was transferred per Rachelle \ndeal dire...
1,Chaves Still,Baric Law,California,NaN,Precision Medical Products,03/23/2023,Med Device 1,NaN,PENDING,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,Declined,Mgr: Jonathan 833-467-2022,Premise,VETTING \nsent vetting emailwith new bill-HD \...


In [16]:
# Antes de eliminar variables irrelevantes eliminamos los registros que no tienen fecha de pago
df = df.dropna(subset=['Pay Dt'])

In [17]:
# Data type casting (corrections)
categorical_columns = ['Account Name','Provider Client','Funding Status','Subject']
df[categorical_columns] = df[categorical_columns].astype('category')

numerical_columns = ['Bill Amt', 'Purch Amt','Pay Amt']
df[numerical_columns] = df[numerical_columns].astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4240 entries, 26 to 8271
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          4240 non-null   object  
 1   Account Name       4240 non-null   category
 2   State              74 non-null     object  
 3   Rate               828 non-null    object  
 4   Provider Client    4228 non-null   category
 5   DOL                3315 non-null   object  
 6   Subject            4240 non-null   category
 7   Name of Procedure  3528 non-null   object  
 8   Case Stage         4240 non-null   object  
 9   Stage Chg Dt       4234 non-null   object  
 10  DOS                4227 non-null   object  
 11  Batch Dt           4204 non-null   object  
 12  Bill Amt           4239 non-null   float64 
 13  Purch Amt          4239 non-null   float64 
 14  Purch Dt           4219 non-null   object  
 15  Investor 1         4238 non-null   object  
 16  Port 1    

In [18]:
df['Account Name'] = (
    df['Account Name']
    .astype(str)
    .str.strip()
    # .str.lower()
    .str.replace(r'\s+', ' ', regex=True)          # Espacios múltiples → uno
    .str.replace(r'[^\w\s&]', '', regex=True)      # Eliminar símbolos raros (excepto &)
    .str.strip()
)
# Creación de columna de limpieza
df['AccountName_Clean'] = np.nan

In [19]:
unique_clean = []

from rapidfuzz import fuzz

for firm in df['Account Name'].unique():
    clean = firm.strip()
    match_found = False
    for ref in unique_clean:
        if fuzz.ratio(clean, ref) > 80:   # Umbral de similitud
            match_found = True
            df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = ref
            break
    if not match_found:
        unique_clean.append(clean)
        df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = clean

/tmp/ipykernel_10861/3216805034.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'A&T Legal Group' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Account Name'] == firm, 'AccountName_Clean'] = clean


In [20]:
# --- Reemplazar columna original por la limpia ---
df['Account Name'] = df['AccountName_Clean']
df = df.drop('AccountName_Clean', axis=1)

print(list(df['Account Name'].unique()))

['A&T Legal Group', 'Aaron Law Group', 'Accident Fighters APC', 'Accident Lawyers Group Pros', 'Ace Law Firm  California', 'Ace Law Group', 'ACTS Law Firm', 'Adamson Ahdoot LLP', 'Agwara Law', 'Alexandroff Law Group', 'Allen Injury Law LLC', 'Allison Law Firm', 'Angulo Law Group', 'Anthony Paglia', 'Ashton & Price LLP', 'Atkinson Watkins & Hoffman Attorneys', 'Avery Injury Law', 'Avrek Law', 'Ayala Morgan & Buzzard', 'BANAFSHE LAW FIRM PC', 'Baradat & Paboojian', 'BD & J Law Firm', 'Benjamin Nadig Law', 'Benson Allred Injury Law', 'Berg Law  California', 'Bertoldo Carter Smith & Mullen Attorneys', 'Beverly Law', 'Bighorn Law  Las Vegas', 'Blackburn Wirth Injury Team', 'Boley & Aldabbagh Attorneys', 'Bonakdar Law Firm', 'Boohoff Law', 'Bruning Law Firm', 'Burger Law', 'Burris & Thomas Law', 'Caffey Law Firm', 'California Accident Law Firm LOP', 'Call Jacob  The Law Offices of Jacob Emrani', 'Cardenas Law Group', 'Carpenter & Zuckerman', 'CARTER WOLDEN CURTIS LLP', 'Christiansen Trial La

In [21]:
df = df[df['Account Name'].map(df['Account Name'].value_counts()) > 2]
print(list(df['Account Name'].unique()))

['Ace Law Group', 'Adamson Ahdoot LLP', 'Agwara Law', 'Allen Injury Law LLC', 'Angulo Law Group', 'Atkinson Watkins & Hoffman Attorneys', 'Ayala Morgan & Buzzard', 'BANAFSHE LAW FIRM PC', 'BD & J Law Firm', 'Benjamin Nadig Law', 'Benson Allred Injury Law', 'Berg Law  California', 'Beverly Law', 'Bighorn Law  Las Vegas', 'Blackburn Wirth Injury Team', 'Bruning Law Firm', 'Burris & Thomas Law', 'Cardenas Law Group', 'Carpenter & Zuckerman', 'Compass Law Group', 'Connell Law', 'David W Fassett Personal Injury Law', 'Dias Hall Law', 'Dimopoulos Injury Law', 'Downtown LA Law Group', 'ER Injury Attorneys', 'Farahi Law Firm', 'Fuller Law Practice', 'G Dallas Horton & Associates', 'Garg Golden Law Firm', 'Gina Corena Law', 'Goldberg Injury Law', 'Hale Injury Law', 'Harris Personal Injury Lawyers  CA', 'Jacoby & Meyers  CA', 'Kang & Associates', 'Karns & Karns Law Firm', 'Ladah Law', 'Lalezary Law Firm  CA Law Brothers', 'Larian Law Firm', 'Larry H Parker Law', 'Law Office of Arash Khorsandi', 

In [22]:
len(list(df['Account Name'].unique()))
# Quedaron un total de 251 categorías

85

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4036 entries, 42 to 8231
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          4036 non-null   object  
 1   Account Name       4036 non-null   object  
 2   State              39 non-null     object  
 3   Rate               767 non-null    object  
 4   Provider Client    4025 non-null   category
 5   DOL                3162 non-null   object  
 6   Subject            4036 non-null   category
 7   Name of Procedure  3500 non-null   object  
 8   Case Stage         4036 non-null   object  
 9   Stage Chg Dt       4030 non-null   object  
 10  DOS                4026 non-null   object  
 11  Batch Dt           4013 non-null   object  
 12  Bill Amt           4035 non-null   float64 
 13  Purch Amt          4035 non-null   float64 
 14  Purch Dt           4015 non-null   object  
 15  Investor 1         4034 non-null   object  
 16  Port 1    

In [24]:
df.head()

,Case Name,Account Name,State,Rate,Provider Client,DOL,Subject,Name of Procedure,Case Stage,Stage Chg Dt,...,Inv 1 Pd Dt,Inv 1 Pd $,Inv 2 Pd Dt,Inv 2 Pd $,Inv 3 Pd Dt,Inv 3 Pd $,Funding Status,Case Notes,Lien Notes,Case Status
42,Sundae McGrath,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,09/06/2018,SMcGrath1,MRI Lumbar Spine wo Contrast,Completed,01/21/2025,...,01/20/2025,907.5,NaN,0.0,NaN,0.0,Purchased,NaN,"02,20 Settled",request has been sent to the finance departmen...
43,Sumala Chuencharoewong,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,SChuen1,MRI Cervical Spine wo Contrast,Completed,09/28/2023,...,05/27/2020,940.5,NaN,NaN,NaN,NaN,Purchased,NaN,NaN,Payment DT 05-27-2020 \n
44,Chai Chuencharoenwong,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,CChuench1,MRI Cervical Spine wo Contrast,Completed,09/27/2023,...,05/27/2020,990.0,NaN,NaN,NaN,NaN,Purchased,NaN,NaN,payment dt 05-27-2020 \n
45,Derek Bell,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,Lien1,MRI Joint Upper Extremity wo Dye,Completed,01/13/2022,...,06/21/2021,825.0,NaN,0.0,NaN,0.0,Purchased,NaN,"02,20 Settled","2/20/20: Settled per case mgr, \nPaid complete..."
46,Derek Bell,Ace Law Group,NaN,NaN,Pueblo Medical Imaging,NaN,Lien2,MRI Cervical Spine wo Contrast,Completed,01/13/2022,...,06/21/2021,825.0,NaN,0.0,NaN,0.0,Purchased,NaN,NaN,"2/20/20: Settled per case mgr, \nPaid complete..."


In [ ]:
df = df.dropna(subset=['Purch Amt'])
df = df[df['Purch Amt'] != 0]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4035 entries, 42 to 8231
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Case Name          4035 non-null   object  
 1   Account Name       4035 non-null   object  
 2   State              39 non-null     object  
 3   Rate               767 non-null    object  
 4   Provider Client    4024 non-null   category
 5   DOL                3162 non-null   object  
 6   Subject            4035 non-null   category
 7   Name of Procedure  3500 non-null   object  
 8   Case Stage         4035 non-null   object  
 9   Stage Chg Dt       4029 non-null   object  
 10  DOS                4026 non-null   object  
 11  Batch Dt           4013 non-null   object  
 12  Bill Amt           4035 non-null   float64 
 13  Purch Amt          4035 non-null   float64 
 14  Purch Dt           4015 non-null   object  
 15  Investor 1         4033 non-null   object  
 16  Port 1    

In [27]:
df.to_excel("Portfolio Puma MGM.xlsx", index=False)